<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-10 14:47:55
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 17.66 K (0.13%)
Current PnL: -12.81 L (-8.92%)
CY Booked + Current PnL: -5.85 L (-4.07%)
-------------------
Total profit:  4.93 L
Total loss:  -17.73 L
-------------------
Total Booked + Current PnL: 21.13 L (18.13%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.29 L (51.92%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87555.0,-7761.0,7801.0,2.50,-8.14,8.91,0.04,157.0,-0.99,0.64,72.69,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-21.55,H-LC,73.96,108600.0,4288.0,13249.0,0.71,4.11,12.20,16.81,27.0,0.32,0.79,20.82,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-18.73,H-LC,88.54,110715.0,7149.0,13563.0,-1.18,6.90,12.25,20.00,32.0,0.53,0.81,21.95,X40N,BTT,HEALTHCARE
71,SBIN,760.30,863.00,-11.51,M-LC,40.62,214902.0,13422.0,13797.0,-0.21,6.66,6.42,13.51,77.0,0.97,1.57,20.10,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,7.29,128898.0,-13581.0,14205.0,0.42,-9.53,11.02,0.44,247.0,-0.96,0.94,52.14,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,41.67,151725.0,812.0,59643.0,0.04,0.54,39.31,40.06,103.0,0.01,1.11,24.1,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,36.46,100043.0,-461.0,31103.0,1.36,-0.46,31.09,30.49,97.0,-0.01,0.73,31.8,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.00,-21.05,M-SC,49.48,164384.0,-3391.0,59228.0,6.72,-2.02,36.03,33.28,206.0,-0.06,1.20,27.27,XY24,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.00,-169.32,M-SC,86.46,79950.0,-22260.0,115648.0,4.18,-21.78,144.65,91.37,212.0,-0.19,0.58,29.78,XY25,NTT,FINANCE
65,RELAXO,902.64,1176.00,-26.70,H-SC,96.88,66246.0,-49292.0,84278.0,3.61,-42.66,127.22,30.28,131.0,-0.58,0.48,29.42,X40N,NTT,FOOTWEAR
54,KPIGREEN,497.21,731.97,31.10,H-SC,72.92,134316.0,9019.0,50140.0,3.03,7.20,37.33,47.22,143.0,0.18,0.98,68.04,MH,ATH,POWER
8,AWL,325.39,485.00,-67.95,H-MC,25.00,222750.0,-45697.0,177376.0,2.76,-17.02,79.63,49.05,98.0,-0.26,1.62,14.53,XY24,NTT,FMCG


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,209.43,M-SC,77.08,138055.0,-45023.0,124001.0,-2.36,-24.59,89.82,43.14,217.0,-0.36,1.01,29.86,XY24,NTT,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,15.62,48432.0,7177.0,3628.0,-1.75,17.40,7.49,26.19,272.0,1.98,0.35,14.02,XY24,ATH,IT
50,JCHAC,2282.24,2282.00,19704.35,M-SC,68.75,96779.0,-31026.0,31008.0,-1.60,-24.28,32.04,-0.01,222.0,-1.00,0.71,9.59,OX40N,NTT,AC
5,ASIANPAINT,2961.56,4250.00,-15.72,H-LC,94.79,167443.0,-33943.0,121564.0,-1.46,-16.85,72.60,43.51,31.0,-0.28,1.22,16.56,X40,BTT,PAINTS
62,PGHH,13388.00,18062.58,-21.40,X-MC,31.25,198300.0,-2520.0,72637.0,-1.46,-1.25,36.63,34.92,55.0,-0.03,1.45,3.36,X40,ATH,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.42,H-SC,98.96,87555.0,-7761.0,7801.0,2.50,-8.14,8.91,0.04,157.0,-0.99,0.64,72.69,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,7.29,128898.0,-13581.0,14205.0,0.42,-9.53,11.02,0.44,247.0,-0.96,0.94,52.14,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.61,M-SC,32.29,152052.0,-19089.0,19037.0,0.63,-11.15,12.52,-0.03,191.0,-1.00,1.11,9.28,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,90.62,148800.0,-25759.0,25787.0,-0.02,-14.76,17.33,0.02,133.0,-1.00,1.08,29.03,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.15,X-MC,6.25,85841.0,-16811.0,16808.0,-0.33,-16.38,19.58,-0.00,59.0,-1.00,0.63,30.49,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,30.21,264248.0,-31686.0,125597.0,0.12,-10.71,47.53,31.73,1.0,-0.25,1.93,5.77,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,59.38,292432.0,30831.0,114780.0,-0.31,11.79,39.25,55.67,2.0,0.27,2.13,18.61,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,62.50,270066.0,-281.0,34920.0,-0.50,-0.10,12.93,12.81,5.0,-0.01,1.97,12.95,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.95,X-LC,78.12,270549.0,41316.0,25973.0,-0.01,18.02,9.60,29.35,10.0,1.59,1.97,28.55,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-5.67,X-LC,11.46,79772.0,-30400.0,119475.0,-0.85,-27.59,149.77,80.85,11.0,-0.25,0.58,17.49,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3130.4,NaN,NaN,95.83,78260.0,43811.0,0.0,0.12,127.18,0.00,127.18,274.0,inf,0.57,19.67,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-13.49,L-SC,100.00,173497.0,31655.0,24914.0,2.41,22.32,14.36,39.88,262.0,1.27,1.26,75.83,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.90,X-SC,91.67,251363.0,45886.0,35442.0,1.85,22.33,14.10,39.58,65.0,1.29,1.83,34.70,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,60.42,232418.0,7456.0,160206.0,0.81,3.31,68.93,74.53,189.0,0.05,1.69,34.94,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.97,31.10,H-SC,72.92,134316.0,9019.0,50140.0,3.03,7.20,37.33,47.22,143.0,0.18,0.98,68.04,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,35.42,117362.0,15358.0,29669.0,0.02,15.06,25.28,44.14,126.0,0.52,0.86,33.74,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-3.77,M-LC,79.17,164455.0,13510.0,95943.0,-0.95,8.95,58.34,72.51,71.0,0.14,1.20,13.28,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-4.20,H-SC,93.75,179800.0,15130.0,53958.0,-0.64,9.19,30.01,41.95,124.0,0.28,1.31,17.34,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3130.40,NaN,NaN,95.83,78260.0,43811.0,0.0,0.12,127.18,0.00,127.18,274.0,inf,0.57,19.67,AR,ATH,ELECTRICAL
54,KPIGREEN,497.21,731.97,31.10,H-SC,72.92,134316.0,9019.0,50140.0,3.03,7.20,37.33,47.22,143.0,0.18,0.98,68.04,MH,ATH,POWER
94,WHIRLPOOL,1167.49,2270.00,-40.70,M-SC,54.17,195481.0,23860.0,138205.0,-1.34,13.90,70.70,94.43,201.0,0.17,1.42,44.28,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,79.17,164455.0,13510.0,95943.0,-0.95,8.95,58.34,72.51,71.0,0.14,1.20,13.28,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,80.21,137355.0,14019.0,119746.0,0.15,11.37,87.18,108.45,123.0,0.12,1.00,36.26,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.78,NaN,NaN,71.88,19005.0,-30588.0,0.0,0.87,-61.68,0.00,-61.68,270.0,-inf,0.14,27.45,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,20.83,67046.0,-46503.0,86556.0,1.66,-40.95,129.10,35.27,265.0,-0.54,0.49,64.10,XR,NTT,HOTELS
32,ENRIN,1377.95,3130.40,NaN,NaN,95.83,78260.0,43811.0,0.0,0.12,127.18,0.00,127.18,274.0,inf,0.57,19.67,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7222.22,M-SC,64.58,80362.0,-13448.0,90062.0,2.30,-14.34,112.07,81.67,245.0,-0.15,0.59,55.63,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87555.0,-7761.0,7801.0,2.50,-8.14,8.91,0.04,157.0,-0.99,0.64,72.69,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,173497.0,31655.0,24914.0,2.41,22.32,14.36,39.88,262.0,1.27,1.26,75.83,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87555.0,-7761.0,7801.0,2.50,-8.14,8.91,0.04,157.0,-0.99,0.64,72.69,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.26,X-SC,97.92,99846.0,-1117.0,95652.0,-0.12,-1.11,95.80,93.63,67.0,-0.01,0.73,48.72,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-26.70,H-SC,96.88,66246.0,-49292.0,84278.0,3.61,-42.66,127.22,30.28,131.0,-0.58,0.48,29.42,X40N,NTT,FOOTWEAR
32,ENRIN,1377.95,3130.40,NaN,NaN,95.83,78260.0,43811.0,0.0,0.12,127.18,0.00,127.18,274.0,inf,0.57,19.67,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,41.95
2,50,70.62


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.74
LC    33.69
MC    21.56
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.12
X40      12.88
XY25     12.05
X40N     11.92
XR        9.97
OX40N     8.21
AR        8.17
MH        1.71
X5K       1.41
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.59
M-SC    15.92
H-LC    14.00
X-LC    12.85
H-MC     9.59
X-MC     7.57
M-LC     5.80
X-SC     4.57
M-MC     4.04
L-SC     2.66
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.22,-0.20,33.71
IT,12.03,-10.18,64.28
FINANCE,8.56,-11.07,51.37
BANKS,7.64,-8.89,53.33
MISC,5.88,-19.44,75.08
PAINTS,5.65,-12.60,34.79
ELECTRICAL,5.45,1.54,44.96
HEALTHCARE,4.75,1.01,27.91
AUTO,4.31,-14.41,64.72


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2620434.0,26
XR,1092243.0,13
AR,1027681.0,10
X40,686514.0,10
X40N,529099.0,11
XY25,465463.0,8
OX40N,305199.0,11
SR,177983.0,2
MH,81243.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1960714.0,20
M-SC,1759464.0,21
H-LC,619169.0,12
X-LC,546493.0,8
H-MC,517997.0,8
M-MC,415425.0,3
X-MC,414996.0,6
X-SC,291226.0,5
M-LC,265465.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      887138.0      6
M-SC       XY24      799802.0      7
H-SC       AR        522207.0      3
M-SC       XR        340308.0      4
M-MC       XY24      299037.0      2
H-MC       XY24      293811.0      3
X-LC       X40       275318.0      3
H-SC       XR        214707.0      3
           X40N      201517.0      4
H-LC       X40N      198917.0      4
M-SC       AR        187764.0      2
           OX40N     164558.0      5
H-LC       X40       159171.0      3
X-MC       X40       156939.0      2
M-LC       XY24      155725.0      2
L-SC       XR        149714.0      2
X-SC       XR        121304.0      1
X-LC       AR        119475.0      1
M-MC       XR        116388.0      1
M-SC       XY25      115648.0      1
X-MC       XY25      105434.0      1
X-LC       XY25      101369.0      3
M-LC       XR         95943.0      1
X-SC       SR         95652.0      1
H-SC       OX40N      85005.0      3
X-MC       XY24       82830.0      1
M-SC       SR         82331.0      1
H-MC       XY25       82058.0      1
H-LC       AR         79709.0      1
           X5K        74579.0      2
M-SC       X40        69053.0      1
H-LC       X200       68686.0      1
H-MC       AR         59643.0      1
L-SC       AR         58883.0      1
L-MC       XR         53879.0      1
X-MC       X40N       52985.0      1
X-LC       X40N       50331.0      1
H-SC       MH         50140.0      1
L-LC       XY25       47157.0      1
X-SC       OX40N      38828.0      2
H-LC       XY24       38107.0      1
X-SC       XY24       35442.0      1
H-MC       MH         31103.0      1
           X40        26033.0      1
           X40N       25349.0      1
L-SC       XY24       24914.0      1
X-MC       OX40N      16808.0      1
M-LC       XY25       13797.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
